<img src="PEST++V3_cover.jpeg" style="float: left">

<img src="flopylogo.png" style="float: right">

<img src="AW&H2015.png" style="float: center">

# Comparing Local Sensitivity to Global Sensitivity Analysis (GSA)

Sensitivity methods we've looked at so far only evaluate the "local" sensitivity at a single set of parameter values.  For example, the Jacobian matrix represents perturbations to a single set of parameter values.  This local view can be a problem in cases when our inverse problem is nonlinear, which means the parameter sensitivities can change depending on what the parameter value is. In contrast, Global Sensitivity Analyses are statistical approaches that characterize how model parameters affect model outputs over a __wide range__ of acceptable parameter values. Because we often test a number of parameter sets during parameter estimation, GSA methods provide a more robust view of parameter sensitivity compared to traditional, derivative-based local sensitivity analysis. Some GSA methods provide general information about the variability of the sensitivities and have relatively low computational requirements, whereas others provide detailed information on nonlinear behavior and interactions between parameters at the expense of larger computational requirements. For a complete introduction to GSA theory and methods, see Saltelli and others (2004, 2008).

### Local Sensitivity Equation
Recall the local sensitivity equation (eq. 9.7 Anderson et al. 2015):

<img src="Sensitivity_eq.png" style="float: center">

In GSA the change in parameter is not just small perturbations (usually 1%) around the parameter value but rather spans the reasonable parameter range.  What is a reasonable range?  For convenience GSA++ uses the parameter bounds that you specify in your PEST Control File.

Anothe way to look at this finite-difference approximation is:

For each parameter-observation combination, we can see how much the observation value changes due to a small change in the parameter. If $y$ are the observations and $x$ are the parameters, the equation for the $i^th$ observation with respect to the $j^th$ parameter is:  
## $\frac{\partial y_i}{\partial x_j}$
This can be approximated by finite differences as :  
## $\frac{\partial y_i}{\partial x_j}~\frac{y\left(x+\Delta x \right)-y\left(x\right)}{\Delta x}$

### But we only do this around one set of parameter values.  What if we looked at more than one set of parameters?

### That's what GSA does, but lots of parameters evaluated = lots of computer time.  Luckily we can use the PEST++ engine to run GSA in parallel

GSA++ is a stand-alone program distributed with the PEST++ Version 3 suite of tools that perform GSA. It leverages the PEST++ parallel run manger YAMR, and is fully compatible with PEST/PEST++ file formats, including template and instruction files. For us this means we can access the insight of GSA using parallel computing without changing our files. More GSA++ running details are provided in appendix 6 of the PEST++ V3 Techniques and Methods Report. Appendixes 7 and 8 demonstrate the use of GSA++ for a Method of Morris and Sobol example problem.  In this notebook we'll run GSA on the Freyburg model.

### Brief take on GSA Theory from the PEST++ T&M Report (Welter et al. 2015 pages 5-7):

The program GSA++ currently supports two GSA methods: (1) the __Method of Morris__ (Morris, 1991), and (2) the __Method of Sobol__ (Sobol, 2001). In the class we'll focus on the Method of Morris because it is computationally more efficient. But this efficiency comes with a tradeoff: the Method of Morris only provides estimates of the mean and variance of the sensitivity distribution for each parameter. Because of the lack of complete description of the parameter nonlinearity and interactions between parameters, the Method of Morris can be used as a screening-level tool to identify the most important parameters for the observations tested. This screening can be followed by application of a more comprehensive tool, such as the Method of Sobol, which further characterizes the effects of parameter nonlinearity and inter-parameter interactions. 

The Method of Morris is referred to as a “one-at-a-time” method because each parameter is perturbed sequentially to compute sensitivities, which is ideally suited for parallel computing. The method samples the sensitivity of a given parameter at several locations over the range of reasonable parameter space (__defined by the parameter bounds in the PEST Control file__) and then provides two measures of parameter sensitivity: the mean (__μ__) and the standard deviation (__σ__) of the resulting sensitivity distribution. The mean, __μ__, captures the overall effect of a parameter on the model output of interest; the standard deviation, __σ__, measures a parameter’s sensitivity across the range of acceptable parameter values, this being an indicator of how nonlinear a given parameter is and (or) how the parameter interacts with other parameters. It is important to note that the Method of Morris __cannot distinguish__ between parameter nonlinearity and parameter interactions because only the standard deviation of parameter sensitivity is available. 

#### For the class we'll focus on the easiest way to get in the GSA game using pyemu, but this is also likely what you would start with in 95% of the models you evaluate.  For convenience, widely applicable default values are supplied for you by pyemu but can easily be changed by editing the *.gsa* file pyemu creates  (see Appendix 6 in the PEST++ V3 report for descriptions of the *.gsa* variables). 

### The objectives of this notebook are:

1) to calculate local sensitivities for the Freyberg zone model and evaluate them

2) look at the Method of Morris GSA results for the same parameters

In [ ]:
%matplotlib inline
import os
import shutil
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyemu

In [ ]:
import freyberg_setup as fs
pst_name = fs.PST_NAME_ZN
working_dir = fs.WORKING_DIR_ZN
fs.setup_pest_zn()

#### Let's double check what parameters we have in this version of the model using pyemu (you can just look in the PEST control file too....)

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.parameter_data

# It's the zone version of the Freyberg model.  

### We'll be comparing local vs global sensitivity so we have to calculate both.  

# 1) Local Sensitivity

First Let's calculate a single Jacobian by changing the NOPTMAX = -2.  This will need npar+1 runs, or 10+1 = 11 runs. The Jacobian matrix we get is the local-scale sensitivity information

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = -2
pst.write(os.path.join(working_dir,pst_name))
pyemu.helpers.run("pestpp {0}".format(pst_name),cwd=working_dir)

#### Let's echo out the working directory in case you want to look at the files that were just created...

In [ ]:
os.getcwd()

### Okay, let's examing the *local sensitivities* by looking at the local gradients of parameters with respect to observations (the Jacobian matrix from the PEST++ NOPTMAX = -2 run)

We'll use pyemu to do this:

In [ ]:
jco = pyemu.Jco.from_binary(os.path.join(working_dir,pst_name.replace(".pst",".jcb")))
jco_df = jco.to_dataframe()

jco_df = jco_df.loc[pst.nnz_obs_names,:]
jco_df

we can see that some parameters (rch_0) has a large effect on the observations used for calibration.  The future recharge (rch_1) has no effect on the calibration observations, but that makes sense as none of the calibration observations are in that future stress period!

# How about Composite Scaled Sensitivities
As can be seen above, parameter sensitivity for any given parameter is split among all the observations in the Jacobian matrix, but the parameter sensitivity that is most important for parameter estimation is the *total* parameter sensitivity, which reflects contributions from all the observations.  

How to sum the individual sensitivities in the Jacobian matrix in the most meaningful way?  In the traditional, overdetermined regression world, CSS was a popular metric. CSS is Composite Scaled Sensitivitity. It sums the observation *weighted* sensitivity to report a single number for each parameter.

In Hill and Tiedeman (2007) this is calculated as: 
## ${css_{j}=\sqrt{\left(\sum_{i-1}^{ND}\left(\frac{\partial y'_{i}}{\partial b_{j}}\right)\left|b_{j}\right|\sqrt{w_{ii}}\right)/ND}}$

In PEST and PEST++, John Doherty calculates it slightly differently in that scaling by the parameter values happens automatically when the parameter is subjected to a log-transform (and we can see above that all our parameters are logged). This is due to a correction that must be made in calculating the Jacobian matrix and follows from the chain rule of derivatives.  Seems somewhat academic, but let's compare the two:


In [ ]:
sc = pyemu.Schur(jco=os.path.join(working_dir,pst_name.replace(".pst",".jcb")))
css_df = sc.get_par_css_dataframe()
css_df

## Hmm, the relative ranks agree between the two...OOPS! Check out rch_0!  We can see from the Jacobian print out above that rch_0 is the *most* sensitive, but the CSS statistics don't  both say that!  Why might this be?

hint: what is the initial value of rch_0?  What is the log of that initial value?  

##  Okay, let's look at just the PEST CSS and rank/plot it:

In [ ]:
plt.figure(figsize=(8,4))
ax = css_df['pest_css'].sort_values(ascending=False).plot(kind='bar')
ax.set_yscale('log')

# So how do these parameter sensitivities affect the forecasts?  

###  Recall that the sensitivity is calculated by differencing the two model outputs, so any model output can have a sensitivity calculated even if we don't have a measured value.  So, because we included the forecasts as observations we have sensitivities for them in our Jacobian matrix.  Let's use pyemu to pull just these forecasts....

In [ ]:
jco_fore_df = sc.forecasts.to_dataframe()
jco_fore_df

## Note that porosity is 0.000 except for the travel time forecast, which makes sense.  

### Perhaps less obvious is rch_0 - why does it have sensitivity when all the forecasts are in the period that has rch_1 recharge? 

### Now let's consider posterior covariance 

In [ ]:
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

For covariance, very small numbers reflect that the parameter doesn't covary with another.  Does it make sense that rch_1 does not covary with other parameters?

### We can visualize the correlation betwen the two parameters using a correlation coefficient

In [ ]:
R = covar.to_pearson()
plt.imshow(R.df(), interpolation='nearest', cmap='viridis')
plt.colorbar()

As expected, the parameters are correlated perfectly to themselves (1.0 along the yellow diagonal) buth they also can have appreciable correlation to each other, both positively and negatively 

#### Using zone hk_z03, let's look only at the parameters that have correlation > 0.5

In [ ]:
cpar = 'hk_z03'
R.df().loc[cpar][np.abs(R.df().loc[cpar])>.5]

Saying parameters are correlated is really saying that when a parameter changes it has a similar effect on the observations as the other parameter(s). So in this case that means that when hk_z03 increases it has a similar effect on observations as increasing hk_z02.  If we add a new observation type (or less powerfully, a new location) we can break the correlation.  And we've seen this:  adding a flux observation broke the correlation between R and K!

We can use this pyemu picture to interrogate the correlation - here we say plot this but cut out all that correlations under 0.9.  Play with this by putting other numbers between 0.3 and 1.0 and re-run the block below.

In [ ]:
R_plot = R.df().as_matrix().copy()
R_plot[np.abs(R_plot)>0.9] = np.nan
plt.imshow(R_plot, interpolation='nearest', cmap='viridis')
plt.colorbar()

In practice, correlation >0.95 or so becomes a problem for the parameter estimation problem.  

#### 2) Global Sensitivity
### Secondly, let's run GSA, a global sensitivity analysis tool.  By default (no extra settings), GSA will run the Method of Morris with 4 discretization points for each parameter (36 runs) plus the 4 new starting points from the intial conditions (4 runs). That is, 44 runs instead of the 11 needed for the local sensitivity method, so we'll use sweep.

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.',"pestpp-gsa",pst_name,num_slaves=15,master_dir=".")
os.chdir("..")

#### Again, let's echo out the working directory in case you want to look at the files that were just created...

In [ ]:
os.getcwd()

## GSA results

Let's look at a table and plot of the GSA results.  In this case we are looking at the mean sensitivity, and the standard deviation of the sensitivity as you change the starting value in the parameter space.  If the __mean sensitivity is high__ it shows that parameter has higher sensitivity across the parameter space.  If the __standard deviation is low__ than the linear assumptions of FOSM hold (that is, the sensitivity is the similar regardless of starting value).  

In [ ]:
df = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".msn")),skipinitialspace=True)
df.loc[:,"parnme"] = df.pop("parameter_name").apply(lambda x : x.lower().replace("log(",'').replace(')',''))
df.index = df.parnme
df

In [ ]:
df.loc[:,["sen_mean_abs","sen_std_dev"]].plot(kind="bar")

### Interesting!  ``hk_z03`` is more sensitive across parameter space compared to the other HK parameters but hk_z02 is similar (hmm, weren't these two correlated above?)...and it and ``rch_0`` looks to be suffering from non-linearity (and/or interactions with other parameters - Method of Morris can't distinguish the two)!

In [ ]:
arr = np.loadtxt(os.path.join(working_dir,"hk.zones"))
arr = np.ma.masked_where(arr!=3,arr)
c = plt.imshow(arr)